# Integrate own Translator
As mentioned in the previous section, to make this as easy to use as possible, this notebook provide a template for integrating own Translator in the pipeline.

In [ ]:
from flask import Flask, request, Response, jsonify
import json
import spacy

app = Flask(__name__)

print("Loading Custom Translator")


You can modify the process_link method, you get the link of a mention (and also a mention in case its needed).
in the default case, this method, replaces the dbpedia link to a wikipedia link.

Modifying the process method is in this case not needed.

In [ ]:
# translates dbpedia to wikipedia
def process_link(link, mention):
    processed_link = link.replace("http://dbpedia.org/resource/", "https://wikipedia.org/wiki/")
    return processed_link

In [ ]:
def process(document):
    for mention_dict in document['mentions']:
        mention_value = mention_dict['mention']
        link = mention_dict['assignment']['assignment']
        processed_link = process_link(link, mention_value)
        mention_dict['assignment']['assignment'] = processed_link

        for possible_assignment in mention_dict['possibleAssignments']:
            processed_link_possible_assignment = process_link(possible_assignment['assignment'], mention_value)
            possible_assignment['assignment'] = processed_link_possible_assignment

    return document

In [ ]:
@app.route('/', methods=['get', 'post'])
def index():
    print("Incoming request:")
    req = json.loads(request.data)


    document = process(req['document'])

    return jsonify(
            {'document' : document,
            'pipelineConfig' : req['pipelineConfig'],
            'componentId' : req['componentId']}
            )


In [ ]:
class LoggingMiddleware(object):
    def __init__(self, app):
        self._app = app

    def __call__(self, env, resp):
        errorlog = env['wsgi.errors']
        #pprint.pprint(('REQUEST', env), stream=errorlog)

        def log_response(status, headers, *args):
            #pprint.pprint(('RESPONSE', status, headers), stream=errorlog)
            return resp(status, headers, *args)

        return self._app(env, log_response)


# Run at flask startup (https://stackoverflow.com/a/55573732)
with app.app_context():
    pass

You can change the port of the web service here, after running the following code the webservice will be available.

**NOTE**: It only accepts get and post requests, trying to open the web page will result in an error with code 500.

In [ ]:
if __name__ == '__main__':
    # MAKE SURE YOU DON'T USE THE SAME PORT ON THE OTHER PIPELINE COMPONENTS
    port = 5006
    print("Running app... on port: ", port)
    app.wsgi_app = LoggingMiddleware(app.wsgi_app)
    app.run(host='0.0.0.0', port=port)